# Sprawdzanie środowiska

In [ ]:
from platform import python_version

print(python_version())

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Przygotowanie odpowiednich danych

In [ ]:
'''
Dla PoC wykonuje obliczenia dla:
 * '../data/DogsCats'
Folder docelowy:
 * '../data/Photos'
'''

dir_path = '../../data/Photos'
A_folder = 'originals'
B_folder = 'photoshops'

In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import math
from keras.preprocessing.image import load_img
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import random

# fix random bo tak ( ͡° ͜ʖ ͡°)
odp = 42
numpy.random.seed(odp)

# Stałe

In [ ]:
# Wilkości

IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

# Przygotowanie Danych

In [ ]:
'''
Opis danych:
1 - klasa 1 -> Originals
0 - klasa 2 -> Photoshops
''' 

A_folder_list = os.listdir(dir_path + '/' + A_folder)
B_folder_list = os.listdir(dir_path + '/' + B_folder)

filenames = []
categories = []

for filename in A_folder_list:
    categories.append(0)
    filenames.append(dir_path + '/' + A_folder + '/' + filename)

for filename in B_folder_list:
    categories.append(1)
    filenames.append(dir_path + '/' + B_folder + '/' + filename)


df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
# Mieszamy!
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
sample = random.choice(df['filename'])
image = load_img(sample)
plt.imshow(image)

# Obróbka zdjęć

In [ ]:
from skimage.feature import hog
import mahotas
import cv2
from PIL import Image, ImageChops, ImageEnhance

def ft_histogram(image, mask=None):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # chanels: [0,1,2], bo mamy przestrzeń HSV
    # mask: None
    # histSize: [8, 8, 8], bin count, po 8 dla każdego z kanałów
    # ranges : [0, 256, 0, 256, 0, 256], wszystko dla każdego z 3 kanałów
    hist  = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def ft_hog(image):
    image = cv2.resize(image, (64, 64))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features, hog_image = hog(image, block_norm='L2-Hys', pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return hog_features, hog_image


def ft_ela(image):
    image = cv2.resize(image, (32, 32))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(image)
    im.save('tmp.jpg', 'JPEG', quality=90)
    resaved_im = Image.open('tmp.jpg')
    
    ela_im = ImageChops.difference(im, resaved_im)
    
    extrema = ela_im.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)

    return numpy.asarray(ela_im).flatten() / 255, ela_im
    

def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        histogram = ft_histogram(image)
        ela = ft_ela(image)[0]
        hog = ft_hog(image)[0]
        global_feature = numpy.hstack([histogram, ela, hog])
        return global_feature
    except Exception as e:
        print(f'Problem with {image_path}, error_msg: {str(e)}')
    

In [ ]:
sample = random.choice(df['filename'])
image = cv2.imread(sample)
image = cv2.resize(image, IMAGE_SIZE)
plt.imshow(image)

In [ ]:
sample

In [ ]:
ela = ft_ela(image)

plt.imshow(ela[1])
print(f'Kształt: {ela[0].shape}')
print(f'Min: {numpy.amax(ela[0])}')
print(f'Max: {numpy.amin(ela[0])}')

In [ ]:
hog_img = ft_hog(image)

plt.imshow(hog_img[1])
print(f'Kształt: {hog_img[0].shape}')
print(f'Min: {numpy.amax(hog_img[0])}')
print(f'Max: {numpy.amin(hog_img[0])}')

In [ ]:
histogram = ft_histogram(image)

print(f'Kształt: {histogram.shape}')
print(f'Min: {numpy.amax(histogram)}')
print(f'Max: {numpy.amin(histogram)}')

In [ ]:
print(f'Wielkość całego wektora dla pojedynczego zdjęcia: {histogram.shape[0] + ela[0].shape[0] + hog_img[0].shape[0]}')
print(f'\t Histogram({histogram.shape[0]}): {(histogram.shape[0] * 100)/(histogram.shape[0] + ela[0].shape[0] + hog_img[0].shape[0]):.2f}%')
print(f'\t Ela({ela[0].shape[0]}): {(ela[0].shape[0] * 100)/(histogram.shape[0] + ela[0].shape[0] + hog_img[0].shape[0]):.2f}%')
print(f'\t Hog({hog_img[0].shape[0]}): {(hog_img[0].shape[0] * 100)/(histogram.shape[0] + ela[0].shape[0] + hog_img[0].shape[0]):.2f}%')


# Przeliczenie Cech Zdjęć + Kategorii

In [ ]:
g_f = []
l = []

for i in df['filename']:
    data_img = preprocess_image(i)
    g_f.append(data_img)

for i in df['category']:
    if i == 0:
        l.append('original')
    else:
        l.append('photoshop')

In [ ]:
from sklearn.preprocessing import LabelEncoder

print(f'Wektor zdjęć: {numpy.array(g_f).shape}')
print(f'Wektor kategorii słownych: {numpy.array(l).shape}\n')

targetNames = numpy.unique(l)
le = LabelEncoder()
target = le.fit_transform(l)


print(f'Wektor kategorii liczbowych: {target.shape}')
print(f'Max arg z nieskalowanego: {numpy.argmax(g_f[0])}, Min arg z nieskalowanego: {numpy.argmin(g_f[0])}')
print(f'Max z nieskalowanego: {numpy.amax(g_f[0])}, Min z nieskalowanego: {numpy.amin(g_f[0])}\n')

# Zapis/Odczyt

In [ ]:
import h5py

def save(features, labels, dataframe, name):
    h5f_data = h5py.File('data_' + str(name) + '.h5', 'w')
    h5f_data.create_dataset('dataset', data=numpy.array(features))

    h5f_label = h5py.File('labels_' + str(name) + '.h5', 'w')
    h5f_label.create_dataset('dataset', data=numpy.array(labels))

    h5f_data.close()
    h5f_label.close()

    dataframe.to_csv('dataframe_' + str(name) + '.csv')
    
def load(features, labels, dataframe):
    h5f_data  = h5py.File(features, 'r')
    h5f_label = h5py.File(labels, 'r')

    global_features_string = h5f_data['dataset']
    global_labels_string   = h5f_label['dataset']

    global_features = numpy.array(global_features_string)
    global_labels   = numpy.array(global_labels_string)

    h5f_data.close()
    h5f_label.close()
    
    df = pd.read_csv(dataframe, index_col = 0)  
    
    return global_features, global_labels, df
    
save(g_f, target, df, name='Casia')

global_features, global_labels, df = load('data_Casia.h5', 'labels_Casia.h5', 'dataframe_Casia.csv')

In [ ]:
df.head()

In [ ]:
print('Kształt po wczytaniu:')
print(f'\tglobal_features: {global_features.shape}')
print(f'\tglobal_labels: {global_labels.shape}')

In [ ]:
global_labels[0]

In [ ]:
ela = global_features[0][512:3584] * 255
ela = ela.astype(numpy.uint8)
ela = ela.reshape(32, 32, 3)
ela.shape

print(f'Kształt: {ela.shape}')
print(f'Min: {numpy.amax(ela)}')
print(f'Max: {numpy.amin(ela)}')

plt.imshow(Image.fromarray(ela))

In [ ]:
from sklearn.decomposition import PCA

n_Casia = 3000
n_sCasia = 100

if global_features.shape[0] < 1000:
    n_comps = n_sCasia
else:
    n_comps = n_Casia

pca = PCA(n_components=n_comps)
global_features_PCA = pca.fit_transform(global_features)

save(global_features_PCA, target, df, name='Casia_with_PCA')

global_features_PCA, global_labels_PCA, df_PCA = load('data_Casia_with_PCA.h5', 'labels_Casia_with_PCA.h5', 'dataframe_Casia_with_PCA.csv')
del global_features, global_labels, df

In [ ]:
print(f'\t Wektor zdjęć: {global_features_PCA.shape}')
print(f'\t Wektor kategorii słownych: {global_labels_PCA.shape}\n')
print(f'\t Wektor dataframe: {df_PCA.shape}')

In [ ]:
df_PCA.head()

# Funkcję liczące statystyki

In [ ]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score

def countStats(_y_true, _y_pred):
    accuracy = accuracy_score(_y_true, _y_pred, normalize=True)
    precision = precision_score(_y_true, _y_pred, average='binary')
    recall = recall_score(_y_true, _y_pred, average='binary')
    fscore = f1_score(_y_true, _y_pred, average='binary')
    
    return accuracy, precision, recall, fscore

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

def plot_cm(cm, classes):
    plot_confusion_matrix(conf_mat=cm,
                          colorbar=True, 
                          show_absolute=True,
                          show_normed=True,
                          class_names=classes)

In [ ]:
def statistical_analysis(scores, name, clfs, info=False):
    printmd(f'### Analiza statystyczna dla {name}:')
    
    alfa = .05
    t_statistic = numpy.zeros((len(clfs), len(clfs)))
    p_value = numpy.zeros((len(clfs), len(clfs)))
    
    for i in range(len(clfs)):
        for j in range(len(clfs)):
            t_statistic[i, j], p_value[i, j] = ttest_ind(scores[i], scores[j])
    headers = ["SVM linear", "SVM poly", "SVM rbf", "SVM sigmoid"]
    names_column = numpy.array([["SVM linear"], ["SVM poly"], ["SVM rbf"], ["SVM sigmoid"]])
    t_statistic_table = numpy.concatenate((names_column, t_statistic), axis=1)
    t_statistic_table = tabulate(t_statistic_table, headers, floatfmt=".2f")
    p_value_table = numpy.concatenate((names_column, p_value), axis=1)
    p_value_table = tabulate(p_value_table, headers, floatfmt=".2f")
    
    if info:
        printmd("**t-statistic:**")
        print(t_statistic_table)
        printmd("**p-value:**")
        print(p_value_table)
    
    
    advantage = numpy.zeros((len(clfs), len(clfs)))
    advantage[t_statistic > 0] = 1
    advantage_table = tabulate(numpy.concatenate(
        (names_column, advantage), axis=1), headers)

    if info:
        printmd("**Przewaga:**")
        print(advantage_table)
    
    
    significance = numpy.zeros((len(clfs), len(clfs)))
    significance[p_value <= alfa] = 1
    significance_table = tabulate(numpy.concatenate(
        (names_column, significance), axis=1), headers)
    
    if info:
        printmd("**Różnice statystycznie znaczące (alpha = 0.05):**")
        print(significance_table)
    
    
    stat_better = significance * advantage
    stat_better_table = tabulate(numpy.concatenate(
        (names_column, stat_better), axis=1), headers)
    printmd("**Wynik końcowy analizy statystycznej:**")
    print(stat_better_table)

# Funkcje do liczenia modelu

In [ ]:
from sklearn.svm import SVC
from sklearn import clone
import copy
from tabulate import tabulate
from scipy.stats import ttest_ind

def do_the_thing(features, labels, name):
    
    clf_list = {
        "SVM linear": [],
        "SVM poly": [],
        "SVM rbf": [],
        "SVM sigmoid": []
    }
    
    tcm_list = copy.deepcopy(clf_list)
    tAccuracy_list = copy.deepcopy(clf_list)
    tPrecision_list = copy.deepcopy(clf_list)
    tRecall_list = copy.deepcopy(clf_list)
    tFScore_list = copy.deepcopy(clf_list)
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=odp)
    clfs = {
        "SVM linear": SVC(kernel='linear', probability=True, random_state=odp, verbose=True, cache_size=1000),
        "SVM poly": SVC(kernel='poly', probability=True, random_state=odp, verbose=True, cache_size=1000),
        "SVM rbf": SVC(kernel='rbf', probability=True, random_state=odp, verbose=True, cache_size=1000),
        "SVM sigmoid": SVC(kernel='sigmoid', probability=True, random_state=odp, verbose=True, cache_size=1000),
    }
    scores_a = numpy.zeros((len(clfs), 5))
    scores_p = numpy.zeros((len(clfs), 5))
    scores_r = numpy.zeros((len(clfs), 5))
    scores_f = numpy.zeros((len(clfs), 5))
    
    for fold_id, (train_index, test_index) in enumerate(kf.split(features, labels)):
        for clf_idx, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])
            clf.fit(features[train_index], labels[train_index])
            y_pred = clf.predict(features[test_index])

            accuracy, precision, recall, fscore = countStats(labels[test_index], y_pred)
            cm = confusion_matrix(labels[test_index], y_pred)
            scores_a[clf_idx, fold_id] = accuracy_score(labels[test_index], y_pred, normalize=True)
            scores_p[clf_idx, fold_id] = precision_score(labels[test_index], y_pred, average='binary')
            scores_r[clf_idx, fold_id] = recall_score(labels[test_index], y_pred, average='binary')
            scores_f[clf_idx, fold_id] = f1_score(labels[test_index], y_pred, average='binary')

            tAccuracy_list[clf_name].append(accuracy)
            tPrecision_list[clf_name].append(precision)
            tRecall_list[clf_name].append(recall)
            tFScore_list[clf_name].append(fscore)
            tcm_list[clf_name].append(cm)

    printmd(f'# {name}:')
    print(f'\n\nKształt danych:')
    print(f'\t X_train: {features[train_index].shape}')
    print(f'\t X_test: {features[test_index].shape}')
    print(f'\t y_train: {labels[train_index].shape}')
    print(f'\t y_test: {labels[test_index].shape}')
    
    results = []
    
    for elem in clf_list:
        accuracy_m = numpy.mean(tAccuracy_list[elem])
        precision_m = numpy.mean(tPrecision_list[elem])
        recall_m = numpy.mean(tRecall_list[elem])
        fscore_m = numpy.mean(tFScore_list[elem])
        
        accuracy_std = numpy.std(tAccuracy_list[elem])
        precision_std = numpy.std(tPrecision_list[elem])
        recall_std = numpy.std(tRecall_list[elem])
        fscore_std = numpy.std(tFScore_list[elem])
        
        cm = sum(tcm_list[elem])
        
        res = [str(elem), 
               f'{accuracy_m:.3f} ({accuracy_std:.2f})',
               f'{precision_m:.3f} ({precision_std:.2f})',
               f'{recall_m:.3f} ({recall_std:.2f})',
               f'{fscore_m:.3f} ({fscore_std:.2f})',
               f'{cm}']
        results.append(res)

    printmd(f'### Rezultaty:')
        
    headers = ["Kernel", "Accuracy", "Precision", "Recall", "Fscore", "CM"]
    print('\n')
    print(tabulate(results, headers=headers))
    
    with open(f'{name}_result.txt', 'w') as f:
        print(tabulate(results, headers=headers), file=f)
    
    numpy.save(f'{name}_results_a', scores_a)
    numpy.save(f'{name}_results_p', scores_p)
    numpy.save(f'{name}_results_r', scores_r)
    numpy.save(f'{name}_results_f', scores_f)
    
    statistical_analysis(scores_a, name="Accuracy", clfs=["SVM linear", "SVM poly", "SVM rbf", "SVM sigmoid"])
    statistical_analysis(scores_p, name="Precision", clfs=["SVM linear", "SVM poly", "SVM rbf", "SVM sigmoid"])
    statistical_analysis(scores_r, name="Recall", clfs=["SVM linear", "SVM poly", "SVM rbf", "SVM sigmoid"])
    statistical_analysis(scores_f, name="Fscore", clfs=["SVM linear", "SVM poly", "SVM rbf", "SVM sigmoid"])

# Wynik

In [ ]:
do_the_thing(global_features_PCA, global_labels_PCA, name='PCA')